# Retinal Organoid Calcium Imaging Pipeline

This notebook demonstrates the full analysis pipeline for calcium imaging data from retinal organoids.

## Pipeline overview
1. Load OME-TIFF and extract sampling rate
2. Compute variance and activity maps
3. Segment ROIs via watershed on the activity score map
4. Build neuropil annuli and extract traces
5. Compute ΔF/F₀ with neuropil subtraction
6. Extract peak features (ISI, prominence, width)
7. Compute pairwise cross-correlation and lag matrices
8. Build functional + spatial affinity and cluster ROIs
9. Compute magnitude-squared coherence and CSD features
10. Save all outputs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
from tifffile import TiffFile
from scipy.spatial.distance import cdist

from pipeline import (
    get_sampling_rate_hz, activity_image_stream, dog2d,
    segment_rois_from_score2d, make_neuropil_masks, extract_traces,
    dff_from_traces, make_temporal_kernel, matched_filter_traces,
    compute_xcorr_lag_matrix, functional_affinity, spatial_affinity,
    combined_affinity, hclust_auto_clusters_from_affinity,
    extract_peak_features, compute_coherence_features, save_results
)

## 1. Load data

In [ ]:
tifpath = r'path/to/your/recording.ome.tif'

with TiffFile(tifpath) as tif:
    video = tif.series[0].asarray()

fs = get_sampling_rate_hz(tifpath)
print(f'Video shape: {video.shape}, Sampling rate: {fs:.2f} Hz')

plt.figure(figsize=(8, 8))
plt.imshow(video[0], cmap='gray')
plt.title('First frame')
plt.show()

## 2. Compute variance and activity score maps

In [ ]:
total_varframe = np.zeros_like(video[0], dtype=np.float32)
window_size = (100, 100)
slide_step = 100
for y in range(0, video.shape[1] - window_size[0] + 1, slide_step):
    y_a = min(y, video.shape[1] - window_size[0])
    y_b = y_a + window_size[0]
    for x in range(0, video.shape[2], slide_step):
        x_a = min(x, video.shape[2] - window_size[1])
        x_b = x_a + window_size[1]
        total_varframe[y_a:y_b, x_a:x_b] = np.var(video[:2500, y_a:y_b, x_a:x_b], axis=0)

S = activity_image_stream(video, fs, hp_s=2.0, chunk_T=512)
S_f = dog2d(S, sigma_small=1.0, sigma_large=4.0)
med = np.median(S_f)
mad = np.median(np.abs(S_f - med)) + 1e-6
score2d = (S_f - med) / mad

fig, axes = plt.subplots(1, 2, figsize=(16, 8))
axes[0].imshow(total_varframe, cmap='gray')
axes[0].set_title('Variance image')
axes[1].imshow(score2d, cmap='gray')
axes[1].set_title('Activity score (z-scored DoG)')
plt.tight_layout()
plt.show()

## 3. Segment ROIs and build neuropil masks

In [ ]:
roi_labels = segment_rois_from_score2d(score2d)
neuropil_labels = make_neuropil_masks(roi_labels)

plt.figure(figsize=(10, 10))
plt.imshow(roi_labels, cmap='nipy_spectral')
plt.imshow(neuropil_labels, cmap='nipy_spectral', alpha=0.2)
plt.title(f'{roi_labels.max()} ROIs detected')
plt.show()

## 4. Extract traces and compute ΔF/F₀

In [ ]:
roi_ids, F_corr, F_roi, F_np = extract_traces(video, roi_labels, neuropil_labels, neuropil_scale=0.7)

dff, F0, Fcorr = dff_from_traces(F_roi, F_np, fs=fs,
                                  neuropil_scale=0.7,
                                  baseline_win_s=60,
                                  baseline_percentile=10,
                                  f0_floor=1.0)

k_t = make_temporal_kernel(fs)
spike_score = matched_filter_traces(F_corr, k_t)

plt.figure(figsize=(18, 6))
offset = 0
for i in range(min(10, dff.shape[0])):
    plt.plot(np.arange(dff.shape[1]) / fs, dff[i] + offset, linewidth=0.5)
    offset += np.nanmax(dff[i]) * 1.1
plt.xlabel('Time (s)')
plt.ylabel('ΔF/F₀ (stacked)')
plt.title('Example ΔF/F₀ traces')
plt.show()

## 5. Peak detection and feature extraction

In [ ]:
parameters_np, parameters, peak_indexes = extract_peak_features(dff, prominence_thresh=0.1)
print(f'Features extracted for {dff.shape[0]} ROIs')
print('Parameters:', parameters)

## 6. Cross-correlation, affinity, and clustering

In [ ]:
L, M, C = compute_xcorr_lag_matrix(dff, max_lag=500)

masks = np.array([roi_labels == rid for rid in roi_ids])
centers = np.array([ndi.center_of_mass(m.astype(float)) for m in masks])
D = cdist(centers, centers)

Wt = functional_affinity(L, M, tau_lag=50)
Ws, sigma_pix = spatial_affinity(D)
W = combined_affinity(Wt, Ws, lam=0.7, beta=0.5)
labels, Z, t = hclust_auto_clusters_from_affinity(W, method='complete')

print(f'{len(np.unique(labels))} clusters found')

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
axes[0].imshow(C, cmap='coolwarm', vmin=-1, vmax=1)
axes[0].set_title('Pearson correlation')
axes[1].imshow(L, cmap='coolwarm')
axes[1].set_title('Lag at max cross-correlation (frames)')
axes[2].imshow(W, cmap='viridis')
axes[2].set_title('Combined affinity')
plt.tight_layout()
plt.show()

## 7. Coherence analysis

In [ ]:
nperseg = min(512, video.shape[0])
noverlap = nperseg // 2
score_per_trace, C_band, S_band, GC_band = compute_coherence_features(
    dff, fs=1.0, nperseg=nperseg, noverlap=noverlap
)

print(f'Global coherence (band-averaged): {GC_band:.4f}')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].bar(range(len(score_per_trace)), score_per_trace)
axes[0].set_xlabel('ROI')
axes[0].set_ylabel('Weighted coherence with mean')
axes[0].set_title('Per-ROI coherence score')
axes[1].imshow(C_band, cmap='viridis')
axes[1].set_title('Band-averaged coherence matrix')
plt.tight_layout()
plt.show()

## 8. Save all results

In [ ]:
out_dir = save_results(
    tifpath, video, total_varframe, score2d,
    roi_labels, neuropil_labels,
    F_roi, F_np, F_corr, dff, labels,
    peak_indexes, parameters_np, parameters,
    C, L, M, D, W,
    score_per_trace, C_band, S_band
)
print(f'Results saved to: {out_dir}')